In [1]:
pip install contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 8.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install rouge

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import contractions
import unicodedata
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, GPT2Model, AdamW, get_linear_schedule_with_warmup
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from rouge import Rouge

2024-04-22 13:57:33.296984: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-22 13:57:33.297093: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-22 13:57:33.408228: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/
nltk.download('punkt')
nltk.download('wordnet')

Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: /usr/share/nltk_data/corpora/wordnet/
  inflating: /usr/share/nltk_data/corpora/wordnet/lexnames  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/adv.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/LICENSE  
  inflating: /usr/share/nltk_data/corpora/wordnet/citation.bib  
  inflating: /usr/share/nltk_data/corpora/wordnet/noun.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/verb.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/README  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.sense  
  inflating: /usr/share/nltk_data

True

In [ ]:
df = pd.read_csv("/kaggle/input/reviewsdata/Reviews.csv")
df.head()

In [ ]:
df.shape

In [ ]:
# Pre-processing functions
def preprocess_text(text):
    if pd.isna(text):
        return ''
    
    # Convert to lowercase
    text = text.lower()
    
    # Remove HTML tags
    # text = re.sub(r'<[^>]+>', '', text)
    clean = re.compile('<.*?>')
    text = re.sub(clean, '', text)

    # Remove accented characters
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')

    # expand contractions in text
    text = contractions.fix(text)

    # remove special characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Tokenize text
    tokens = word_tokenize(text)

    # Lemmatize tokens
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # Join tokens back into text
    preprocessed_text = ' '.join(lemmatized_tokens)

    return preprocessed_text

In [ ]:
# Apply preprocessing to text columns in DataFrame
df['Text'] = df['Text'].apply(preprocess_text)
df['Summary'] = df['Summary'].apply(preprocess_text)

In [5]:
import pickle

In [ ]:
with open("dataframe.pkl", "wb") as f:
    pickle.dump(df, f)
with open("dataframe.pkl", "rb") as f:
    df = pickle.load(f)

In [6]:
with open("/kaggle/input/dataframe/dataframe.pkl", "rb") as f:
    df = pickle.load(f)

In [7]:
df.shape

(568454, 10)

In [9]:
df = df.iloc[:20000,:]

In [10]:
# Divide the dataset into training and testing (75:25)
train_df, test_df = train_test_split(df, test_size=0.25, random_state=42)

In [11]:
class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length):
        self.df = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length
        # Set padding token explicitly
        self.tokenizer.pad_token = self.tokenizer.eos_token  # Set padding token to end of sequence token
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        review_text = self.df.iloc[idx]['Text']
        summary_text = self.df.iloc[idx]['Summary']
        
        # Combine review text and summary text
        text = "Review Text: " + review_text + "\nSummary: " + summary_text
        
        # Tokenize the combined text
        inputs = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        input_ids = inputs['input_ids'].squeeze(0)
        attention_mask = inputs['attention_mask'].squeeze(0)
        
        # Convert score to tensor
        label = torch.tensor(self.df.iloc[idx]['Score'])
        
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'label': label
        }

In [12]:
# Instantiate GPT-2 tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [13]:
# Define dataset and dataloader
train_dataset = CustomDataset(train_df, tokenizer, max_length=128)
train_loader = DataLoader(train_dataset, batch_size=15, shuffle=True)

In [15]:
# Define optimizer and scheduler
learning_rate = 5e-5
epochs = 25
warmup_steps = int(0.1 * len(train_loader) * epochs)
optimizer = AdamW(model.parameters(), lr=learning_rate)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=len(train_loader) * epochs)

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [16]:
# Fine-tuning loop
model.config.pad_token_id = model.config.eos_token_id
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.train()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [17]:
for epoch in range(epochs):
    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=input_ids)
        loss = outputs.loss
        # Backward pass
        loss.backward()
        # Update weights
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")

# Save the fine-tuned model
model.save_pretrained("fine_tuned_gpt2")

Epoch 1/25, Loss: 2.4026856422424316
Epoch 2/25, Loss: 1.9614362716674805
Epoch 3/25, Loss: 2.193612813949585
Epoch 4/25, Loss: 1.8462305068969727
Epoch 5/25, Loss: 1.8588788509368896
Epoch 6/25, Loss: 1.7907968759536743
Epoch 7/25, Loss: 2.3348615169525146
Epoch 8/25, Loss: 2.0684378147125244
Epoch 9/25, Loss: 1.9258642196655273
Epoch 10/25, Loss: 1.749476432800293
Epoch 11/25, Loss: 1.8093793392181396
Epoch 12/25, Loss: 1.6693146228790283
Epoch 13/25, Loss: 1.4474784135818481
Epoch 14/25, Loss: 1.3957884311676025
Epoch 15/25, Loss: 1.4875835180282593
Epoch 16/25, Loss: 1.2850886583328247
Epoch 17/25, Loss: 1.334791660308838
Epoch 18/25, Loss: 1.44439697265625
Epoch 19/25, Loss: 1.4067505598068237
Epoch 20/25, Loss: 1.2036770582199097
Epoch 21/25, Loss: 1.3118188381195068
Epoch 22/25, Loss: 1.2833493947982788
Epoch 23/25, Loss: 1.1619142293930054
Epoch 24/25, Loss: 1.2342007160186768
Epoch 25/25, Loss: 1.2154666185379028


In [18]:
def generate_summary(review_text):
    # Tokenize the review text
    inputs = tokenizer.encode_plus(
        review_text,
        return_tensors="pt",
        max_length=1024,
        truncation=True, 
        padding=True
    )
    
    # Move input tensors to the same device as the model
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    inputs = {key: tensor.to(device) for key, tensor in inputs.items()}

    # Generate summary using the model
    model = GPT2LMHeadModel.from_pretrained('fine_tuned_gpt2').to(device) 
    summary_ids = model.generate(
        inputs['input_ids'], 
        attention_mask=inputs['attention_mask'],
        max_length=1024, 
        num_beams=4, 
        early_stopping=True
    )

    # Decode the generated summary tokens
    generated_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    return generated_summary

In [19]:
ReviewText_list = test_df['Text'].tolist()
Summary_list = test_df['Summary'].tolist()

In [20]:
GeneratedSummary = []
for index, text in enumerate(ReviewText_list):
    g_summary = generate_summary(text)
    split_summary = g_summary.split(text)
    if len(split_summary) > 1:
        generated_summary = split_summary[1].strip()
        GeneratedSummary.append(generated_summary)
    else:
        GeneratedSummary.append("")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

In [21]:
with open("Summary_list.pkl", "wb") as f:
    pickle.dump(Summary_list, f)
with open("Summary_list.pkl", "rb") as f:
    Summary_list = pickle.load(f)

In [22]:
with open("ReviewText_list.pkl", "wb") as f:
    pickle.dump(ReviewText_list, f)
with open("ReviewText_list.pkl", "rb") as f:
    ReviewText_list = pickle.load(f)

In [23]:
with open("GeneratedSummary.pkl", "wb") as f:
    pickle.dump(GeneratedSummary, f)
with open("GeneratedSummary.pkl", "rb") as f:
    GeneratedSummary = pickle.load(f)

In [24]:
# Initialize Rouge
rouge = Rouge()

RougeScore_list = []

for index, i in enumerate(Summary_list):  
    # Compute Rouge scores
    if GeneratedSummary[index].strip() != '' and Summary_list[index].strip() != '' and GeneratedSummary[index].strip() != '.' and Summary_list[index].strip() != '.':   
        rouge_scores = rouge.get_scores(GeneratedSummary[index], i)
        RougeScore_list.append(rouge_scores)

In [26]:
for rouge_scores in RougeScore_list:
    # Print Rouge scores
    print("ROUGE-1: Precision: {:.2f}, Recall: {:.2f}, F1-Score: {:.2f}".format(
        rouge_scores[0]['rouge-1']['p'],
        rouge_scores[0]['rouge-1']['r'],
        rouge_scores[0]['rouge-1']['f']
    ))
    print("ROUGE-2: Precision: {:.2f}, Recall: {:.2f}, F1-Score: {:.2f}".format(
        rouge_scores[0]['rouge-2']['p'],
        rouge_scores[0]['rouge-2']['r'],
        rouge_scores[0]['rouge-2']['f']
    ))
    print("ROUGE-L: Precision: {:.2f}, Recall: {:.2f}, F1-Score: {:.2f}".format(
        rouge_scores[0]['rouge-l']['p'],
        rouge_scores[0]['rouge-l']['r'],
        rouge_scores[0]['rouge-l']['f']
    ))
    print("\n\n")

ROUGE-1: Precision: 0.40, Recall: 0.67, F1-Score: 0.50
ROUGE-2: Precision: 0.25, Recall: 0.50, F1-Score: 0.33
ROUGE-L: Precision: 0.40, Recall: 0.67, F1-Score: 0.50



ROUGE-1: Precision: 0.00, Recall: 0.00, F1-Score: 0.00
ROUGE-2: Precision: 0.00, Recall: 0.00, F1-Score: 0.00
ROUGE-L: Precision: 0.00, Recall: 0.00, F1-Score: 0.00



ROUGE-1: Precision: 0.00, Recall: 0.00, F1-Score: 0.00
ROUGE-2: Precision: 0.00, Recall: 0.00, F1-Score: 0.00
ROUGE-L: Precision: 0.00, Recall: 0.00, F1-Score: 0.00



ROUGE-1: Precision: 0.00, Recall: 0.00, F1-Score: 0.00
ROUGE-2: Precision: 0.00, Recall: 0.00, F1-Score: 0.00
ROUGE-L: Precision: 0.00, Recall: 0.00, F1-Score: 0.00



ROUGE-1: Precision: 0.00, Recall: 0.00, F1-Score: 0.00
ROUGE-2: Precision: 0.00, Recall: 0.00, F1-Score: 0.00
ROUGE-L: Precision: 0.00, Recall: 0.00, F1-Score: 0.00



ROUGE-1: Precision: 0.00, Recall: 0.00, F1-Score: 0.00
ROUGE-2: Precision: 0.00, Recall: 0.00, F1-Score: 0.00
ROUGE-L: Precision: 0.00, Recall: 0.00, F1-Score: 

In [27]:
len(RougeScore_list)

1207

In [31]:
with open("RougeScore_list.pkl", "wb") as f:
    pickle.dump(RougeScore_list, f)